In [11]:
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader


- sampling distribution: z~N(O, I)
- posterior model: g_nu(z) = theta
- expected reward model: f_theta(action) = E(y|action, theta)
- noise for observations: sigmao
- prior std: sigmap

In [13]:
def gather_dataset(env, npoints, nbandits):
    """
    output of shape (Npoints, Kbandits)
    """
    return np.array([[env.step(i) for i in range(nbandits)] for _ in range(npoints)])

In [14]:
d = gather_dataset(KBandits(4), 1000, 4)
d.mean(0), d.std(0)

(array([ 0.6109008 , -0.06494404,  0.1155856 ,  0.40743141]),
 array([0.49614104, 0.50321473, 0.49427322, 0.49183457]))

In [15]:
def index_selection_mapping(theta, x):
    """
    theta of size (B, K)
    x of size (B,)
    """
    return torch.gather(theta, dim=1, index=x.unsqueeze(1))

In [16]:
class LinearHypermodelBandits:
    def __init__(self, hpm_size, sigma_prior, k_arms, device='cpu'):
        self.hpm_size = hpm_size
        self.sigma_p = sigma_prior
        self.n_arms = k_arms
        self.posterior_model_g = LinearModuleBandits(k_bandits=k_arms, model_dim=hpm_size)
        self.device = device
        self.posterior_model_g = self.posterior_model_g.to(device)
        self.prior = self.sample_prior_dbz() # of shape (k, )
        
    
    def sample_prior_dbz(self):
        D = np.random.randn(self.n_arms) * self.sigma_p # dim (k, k)
        B = generate_hypersphere(dim=self.hpm_size, n_samples=self.n_arms, norm=1) # dim (k, m)
        z = np.random.randn(self.n_arms, self.hpm_size) # dim (k, m)
        return torch.from_numpy(D * (B * z).sum(-1)).to(self.device)
    
    def sample_posterior(self, n_samples):
        return self.posterior_model_g.sample(n_samples) + self.prior # NEED TO DISCUSS THIS
    
    def update_device(device):
        self.device = device
        self.prior_posterior_model_g = self.prior_posterior_model_g.to(device)

In [25]:
hm = LinearHypermodelBandits(hpm_size=1, sigma_prior=0.5, k_arms=4)

In [26]:
data_test = np.array([hm.sample_prior_dbz().numpy() for _ in range(1000)])

In [27]:
data_test.mean(0), data_test.std(0)

(array([-0.00923772,  0.00077974, -0.00522484,  0.02766258]),
 array([0.46525   , 0.50226105, 0.48135034, 0.49806974]))

In [18]:
class LinearModuleBandits(nn.Module):
    def __init__(self, k_bandits, model_dim):
        super(LinearModuleBandits, self).__init__()
        self.k = k_bandits
        self.m = model_dim
        self.C = nn.Parameter(torch.ones(size=(k_bandits, model_dim)))
        self.mu = nn.Parameter(torch.ones(k_bandits))
        self.init_parameters()
    
    def init_parameters(self):
        mu_sampled = torch.randn(self.k) * 0.05
        c_sampled = torch.randn(self.k, self.m) * 0.05
        self.C.data = c_sampled
        self.mu.data = mu_sampled
    
    def forward(self, z):
        """
        z of size (batch, Kbandits, modelsize)
        theta of size (batch, Kbandits)
        """
        return (self.C * z).sum(-1) + self.mu
    
    def sample(self, n_samples):
        return self.forward(torch.randn(n_samples, self.k, self.m))

In [28]:
data_test = hm.posterior_model_g.sample(10000)
data_test.mean(0), data_test.std(0)

(tensor([ 0.0049, -0.0116,  0.0170,  0.0108], grad_fn=<MeanBackward1>),
 tensor([0.0751, 0.0433, 0.0542, 0.0729], grad_fn=<StdBackward1>))

In [29]:
hm.prior

tensor([-0.0279,  0.0189,  0.2632,  0.0967], dtype=torch.float64)

In [30]:
hm.posterior_model_g.mu.data

tensor([ 0.0051, -0.0109,  0.0166,  0.0106])

In [31]:
data_test = hm.sample_posterior(10000)
data_test.mean(0), data_test.std(0)

(tensor([-0.0219,  0.0084,  0.2799,  0.1084], dtype=torch.float64,
        grad_fn=<MeanBackward1>),
 tensor([0.0738, 0.0435, 0.0538, 0.0727], dtype=torch.float64,
        grad_fn=<StdBackward1>))

In [20]:
def generate_hypersphere(dim, n_samples, norm=1):
    if norm==1: # TODO ask question about that
        samples = np.random.rand(n_samples, dim)
        samples = samples / np.expand_dims(np.abs(samples).sum(1), 1)
        return samples
    elif norm==2:
        samples = np.random.randn(n_samples, dim)
        samples = samples / np.expand_dims(np.sqrt((samples ** 2).sum(1)), 1)
        return samples
    else:
        raise ValueError

In [21]:
class RandomAgent:
    def __init__(self, k_bandits):
        self.n_arms = k_bandits

    def act(self):
        return np.random.randint(self.n_arms)

    def reset(self):
        pass

In [22]:
def augmented_dataset(dataset, perturbations_dimension, mode='hypersphere'):
    n_points = len(dataset)
    if mode == 'hypersphere':
        perturbations = generate_hypersphere(dim=perturbations_dimension, n_samples=n_points, norm=1)
    else:
        perturbations = generate_hypersphere(dim=perturbations_dimension, n_samples=n_points, norm=2)
    return [tuple([*data_point, perturbations[ix, :]]) for ix, data_point in enumerate(dataset)]

In [23]:
def run_episode(envnmt, actor, horizon, n_steps, n_samples_z, lr, sigmao, sigmap, batch_size, hypermodel, update_every=1, training=False, device='cpu'):
    obs = []
    new_data = []
    dataset = []
    for ix in range(horizon):
        arm_selected = actor.act()
        reward = envnmt.step(arm_selected)
        data_point = [arm_selected, reward]
        obs.append(data_point)
        new_data.append(data_point)
        if (ix + 1) % update_every == 0:
            dataset += augmented_dataset(new_data, perturbations_dimension=hypermodel.hpm_size)
            new_data = []
            if training:
                data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
                train_hypermodel(data_loader,
                                 nsteps=n_steps,
                                 nsamples_z=n_samples_z,
                                 learning_rate=lr,
                                 hypermodel=hypmodel,
                                 sigmao=sigmao,
                                 sigmap=sigmap,
                                 device='cpu')
    return obs, dataset

In [24]:
def train_hypermodel(data_loader, hypermodel, nsteps, nsamples_z, learning_rate, sigmao, sigmap, device='cpu'):
    optimizer = SGD(hypermodel.posterior_model_g.parameters(), lr=learning_rate, weight_decay=1/(2 * sigmap ** 2))
    steps_done = 0
    total_loss = 0
    while True:
        for batch in data_loader:
            x, y, a = batch
            x = x.to(device)
            y = y.to(device)
            a = a.to(device) # shape B, m
#             import ipdb;
#             ipdb.set_trace()
            z_sample = torch.randn(nsamples_z, k, modelling_size, device=device) # shape M, K, m
            z_sliced = torch.index_select(z_sample, 1, x) # shape M, B, m
            
            sigAz = sigmao * (a.unsqueeze(0) * z_sliced).sum(-1) #shape M, B
            
            posteriors = hypermodel.posterior_model_g(z_sample)
            outputs = torch.index_select(posteriors, 1, x)
            
            loss = (((y + sigAz - outputs) ** 2).mean(1)).mean(0) #/ (2 * sigmao ** 2)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
            
            steps_done += 1
            if (steps_done % 25) == 0:
                print(f"step {steps_done}, loss:{loss.item():2f}")
            if steps_done >= nsteps:
                return total_loss / nsteps

In [32]:
H = 1000
sigmao = 0.1 # environmnet parameters
sigmap = 1.

sigmap_algo = 0.5 # hypermodel prior width

sigmap_training = 10. # weight decay penalty
sigmao_training = 0.5
updates_freq = 1
batch_s = 16
lr = 5 * 1e-2
n_samples_z = 16
n_steps = 1000

k = 10
device = 'cuda' if torch.cuda.is_available() else 'cpu'
modelling_size = 2

In [33]:
env = KBandits(k=k,
               sigma_obs=sigmao,
               sigma_model=sigmap)

In [34]:
env.rewards

array([ 0.37918959, -0.69634992,  0.72583669, -0.92180339,  0.38468225,
       -0.24488638,  0.40237648,  0.36897392, -0.01279609, -0.21216963])

In [35]:
d = gather_dataset(env, 1000, k)
d.mean(0), d.std(0)

(array([ 0.38412976, -0.69290271,  0.72406354, -0.92475952,  0.38947336,
        -0.24202389,  0.39885424,  0.37386838, -0.00576701, -0.2126568 ]),
 array([0.09937621, 0.09730512, 0.10095172, 0.09576024, 0.09575365,
        0.09955051, 0.09890657, 0.10147648, 0.09624598, 0.10038414]))

In [36]:
hypmodel = LinearHypermodelBandits(hpm_size=modelling_size,
                                   sigma_prior=sigmap_algo,
                                   k_arms=k,
                                   device=device)

In [46]:
data_test = hypmodel.posterior_model_g.sample(10000)
data_test.mean(0), data_test.std(0)

(tensor([ 0.3528, -0.6733,  0.7074, -0.8959,  0.3708, -0.2315,  0.4133,  0.3414,
         -0.0192, -0.2151], grad_fn=<MeanBackward1>),
 tensor([0.3452, 0.3431, 0.3481, 0.3429, 0.3458, 0.3482, 0.3461, 0.3516, 0.3437,
         0.3524], grad_fn=<StdBackward1>))

In [47]:
hypmodel.prior

tensor([-0.0261, -0.6305, -0.0487, -0.0297, -0.2935, -0.3449,  0.7815,  0.4746,
         0.0347,  0.6643], dtype=torch.float64)

In [48]:
hypmodel.posterior_model_g.mu.data

tensor([ 0.3591, -0.6776,  0.7099, -0.9018,  0.3716, -0.2324,  0.4161,  0.3399,
        -0.0202, -0.2139])

In [49]:
data_test = hypmodel.sample_posterior(10000)
data_test.mean(0), data_test.std(0)

(tensor([ 0.3339, -1.3027,  0.6554, -0.9324,  0.0847, -0.5825,  1.1973,  0.8038,
          0.0187,  0.4500], dtype=torch.float64, grad_fn=<MeanBackward1>),
 tensor([0.3389, 0.3480, 0.3473, 0.3455, 0.3422, 0.3466, 0.3337, 0.3550, 0.3440,
         0.3537], dtype=torch.float64, grad_fn=<StdBackward1>))

In [41]:
env.rewards

array([ 0.37918959, -0.69634992,  0.72583669, -0.92180339,  0.38468225,
       -0.24488638,  0.40237648,  0.36897392, -0.01279609, -0.21216963])

In [43]:
obs, dataset = run_episode(envnmt=env,
                           actor=RandomAgent(k_bandits=k),
                           horizon=H,
                           n_steps=n_steps,
                           n_samples_z=n_samples_z,
                           lr=lr,
                           sigmao = sigmao_training,
                           sigmap = sigmap_training,
                           batch_size=batch_s,
                           update_every=1,
                           hypermodel=hypmodel,
                           training=False,
                           device='cpu')

In [44]:
dl = DataLoader(dataset, batch_size=batch_s, shuffle=True)

In [45]:
train_hypermodel(data_loader=dl,
                 nsteps=n_steps, 
                 nsamples_z=n_samples_z,
                 learning_rate=lr,
                 sigmao=sigmao_training,
                 sigmap=sigmap_training,
                 hypermodel=hypmodel, device='cpu')

step 25, loss:0.200785
step 50, loss:0.158958
step 75, loss:0.088616
step 100, loss:0.107625
step 125, loss:0.072764
step 150, loss:0.075434
step 175, loss:0.064682
step 200, loss:0.046084
step 225, loss:0.061481
step 250, loss:0.035499
step 275, loss:0.036420
step 300, loss:0.042691
step 325, loss:0.058887
step 350, loss:0.044830
step 375, loss:0.021625
step 400, loss:0.048703
step 425, loss:0.040008
step 450, loss:0.032660
step 475, loss:0.048827
step 500, loss:0.050698
step 525, loss:0.039727
step 550, loss:0.031227
step 575, loss:0.040123
step 600, loss:0.043673
step 625, loss:0.028190
step 650, loss:0.046981
step 675, loss:0.045161
step 700, loss:0.052976
step 725, loss:0.024599
step 750, loss:0.024169
step 775, loss:0.041763
step 800, loss:0.050895
step 825, loss:0.061917
step 850, loss:0.031203
step 875, loss:0.044208
step 900, loss:0.039454
step 925, loss:0.025328
step 950, loss:0.026930
step 975, loss:0.035721
step 1000, loss:0.055033


0.06032814111086119

TODOs:
- online loop with TS
- similar offline tests with the MNL bandit env
- script for experiments